In [1]:
load_ext autoreload

In [2]:
!ln -s /home/app/src/main.py main.py
!ln -s /home/app/src/team.py team.py
!ln -s /home/app/src/position.py position.py

ln: failed to create symbolic link 'main.py': File exists
ln: failed to create symbolic link 'team.py': File exists
ln: failed to create symbolic link 'position.py': File exists


In [3]:
autoreload 2

In [4]:
from main import get_players_df

In [5]:
from datetime import datetime
date_str = datetime.today().strftime('%Y-%m-%d')

In [6]:
players_df = await get_players_df(date_str)

In [7]:
players_df["pp90"] = players_df["total_points"] / players_df["minutes"] * 90

In [8]:
players_df["ppv"] = players_df["total_points"] / players_df["now_cost"]

In [9]:
df = players_df[["name", "now_cost", "team_name", "total_points", "position", "pp90", "bonus", "ppv"]]

In [126]:
df_playing = df[df["pp90"] > 2.0]

In [127]:
len(df_playing)

364

In [129]:
import qgrid
qgrid.show_grid(df_playing)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [130]:
def_df = df_playing[df_playing["position"] == "DEF"]
mid_df = df_playing[df_playing["position"] == "MID"]
fwd_df = df_playing[df_playing["position"] == "FWD"]

In [97]:
import constraint

layout of team. Rightmost player is benched

### Keepers
a b
### Defenders
c d e f g
### Midfielders
h i j k l
### Forwards
m n o

In [178]:
def get_player(df, name):
    idx_array = df[df["name"] == name].index.values
    return idx_array

In [191]:
from collections import Counter


problem = constraint.Problem()
problem.addVariable('A', get_player(df, "Mathew Ryan"))
problem.addVariable('B', get_player(df, "David Button"))

problem.addVariable('C', get_player(df, "Trent Alexander-Arnold"))
problem.addVariable('D', get_player(df, "Matt Doherty"))
problem.addVariable('E', get_player(df, "Charlie Taylor"))
problem.addVariable('F', get_player(df, "James Justin"))
problem.addVariable('G', get_player(df, "Rob Holding"))

problem.addVariable('H', get_player(df, "Kevin De Bruyne"))
problem.addVariable('I', get_player(df, "Bruno Miguel Borges Fernandes"))
problem.addVariable('J', get_player(df, "Willian Borges Da Silva"))
problem.addVariable('K', mid_df.index.values)
problem.addVariable('L', get_player(df, "Adama Traoré"))

problem.addVariable('M', fwd_df.index.values)
problem.addVariable('N', get_player(df, "Aleksandar Mitrović"))
problem.addVariable('O', get_player(df, "Harry Kane"))

def max_three_per_team(*argv):
    teams = [df.loc[[id]]["team_name"].values for id in argv]
    teams_frequency = Counter(teams).values()
    if not any(frequency > 3 for frequency in teams_frequency):
        return True
    
def sum_price(*argv):
    prices = [df.loc[[id]]["now_cost"].values for id in argv]
    if sum(prices) < 1000:
        return True

problem.addConstraint(sum_price, "ABCDEFGHIJKLMNO")
problem.addConstraint(constraint.AllDifferentConstraint())

solutions = problem.getSolutions()

def points(ids):
    return sorted([df.loc[[id]]["total_points"].values for id in ids])

def team_points(ids):
    keepers = points(ids[0:2])
    defenders = points(ids[2:7])
    midfilders = points(ids[7:11])
    strikers = points(ids[12:])
    base_points = sum(keepers[0]) + sum(defenders[0:3]) + sum(strikers[0]) # 5 players
    rest = sorted(defenders[3:] + midfilders + strikers[1:]) # best of the rest 6
    return base_points + sum(rest[0:6])

def get_team_from_dict(df, d):
    d_r = d.copy()
    for key, id in d.items():
        d_r[key] = df.loc[[id]]["name"].values[0]
    return d_r

def get_team_cost_from_dict(df, d):
    return  sum([df.loc[[id]]["now_cost"].values[0] for id in d.values()])

total_points = [team_points(list(d.values())) for d in solutions]
max_points = max(total_points)
print(max_points)
max_points_index = total_points.index(max(max_points))
max_points_solution = solutions[max_points_index]
print(get_team_from_dict(df, max_points_solution))
print(get_team_cost_from_dict(df, max_points_solution))

[1112]
{'A': 'Mathew Ryan', 'B': 'David Button', 'C': 'Trent Alexander-Arnold', 'D': 'Matt Doherty', 'E': 'Charlie Taylor', 'F': 'James Justin', 'G': 'Rob Holding', 'H': 'Kevin De Bruyne', 'I': 'Bruno Miguel Borges Fernandes', 'J': 'Willian Borges Da Silva', 'L': 'Adama Traoré', 'N': 'Aleksandar Mitrović', 'O': 'Harry Kane', 'M': 'Jordan Ayew', 'K': 'Declan Rice'}
995
